In [209]:
import selenium
from selenium import webdriver
import time,re,random,sys
import pandas as pd
from bs4 import BeautifulSoup 
#import requests
#from fake_useragent import UserAgent
#import urllib
# import tools
from selenium.common.exceptions import NoSuchElementException,TimeoutException,WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [287]:
def deal_comments_infos_single(url):
    
    def get_comments(ps):
        comments=[]
        for comment in re.findall('<div class="tm-rate-content"><div class="tm-rate-fulltxt">(.*?)</div>',ps):
            if 'span' not in comment:
                comments.append(comment)
        return pd.DataFrame({
                'comment':comments
        })

    info = []
    if not url.startswith('http'):
        url = 'https:' + url

    print('开始寻找评论from:' + url)
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-automation']) 
    options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    driver = webdriver.Chrome(options = options)
    timeout = 30
    
    try:
        driver.get(url)
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.ID, "J_TabBarBox"))  # 判断页面是否初步加载成功的标记
        )
    except TimeoutException:
        print('宝贝链接未加载成功')
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    title = soup.findAll(name='h1',attrs={'data-spm':"1000983"})[0].text.strip()
    
    try:
        # 页面上拉600，看到TabBarBox
        js = "window.scrollTo(0,600)"
        driver.execute_script(js)
    except WebDriverException:
        print('上拉寻找评论区时出现问题')
        
    time.sleep(random.uniform(1,2))
    # 点击累计评论TAB
    driver.find_element_by_xpath('//a[@href="#J_Reviews"]').click()
    time.sleep(random.uniform(1,2))
    print('成功点击累计评论TAB')
    
    while True:
        try:
            # rate-grid的正则表达式
            driver.find_element_by_xpath('//*[@id="J_Reviews"]/div/div[6]')
            print('已经成功加载到评论区信息')
            break
        except NoSuchElementException:
            print('评论信息中元素还未加载')
            continue
    

    allcmts = pd.DataFrame()
    count = 2
    
    while True:
        cmts = get_comments(driver.page_source)
        allcmts = pd.concat([allcmts,cmts],axis=0,ignore_index=True)

        try:
            js = "window.scrollTo(0,2500)"
            driver.execute_script(js)
            time.sleep(random.uniform(1,2))
            # 点击下一页
            nextpage = driver.find_element_by_css_selector('#J_Reviews > div > div.rate-page > div > a:last-child')
            driver.execute_script('arguments[0].click()',nextpage)
            print('\r已成功点击第' + str(count) +'页',end='')

        except NoSuchElementException:
            print('\n')
            print('找不到翻页按钮,结束查询,(%s)' % title)
            break
            
        count += 1
        time.sleep(random.uniform(1,2))
    
    driver.close()
    return title,allcmts

In [286]:
def deal_comments_infos_fromurlist(urlist):
    
    def get_comments(ps):
        comments=[]
        for comment in re.findall('<div class="tm-rate-content"><div class="tm-rate-fulltxt">(.*?)</div>',ps):
            if 'span' not in comment:
                comments.append(comment)
        return pd.DataFrame({
                'comment':comments
        })
    
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-automation']) 
    options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    driver = webdriver.Chrome(options = options)
    timeout = 30
        
    for url in urlist:
        if not url.startswith('http'):
            url = 'https:' + url
        print('开始寻找评论from:' + url)


        try:
            driver.get(url)
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.ID, "J_TabBarBox"))  # 判断页面是否初步加载成功的标记
            )
        except TimeoutException:
            print('宝贝链接未加载成功')

        soup = BeautifulSoup(driver.page_source, 'lxml')
        title = soup.findAll(name='h1',attrs={'data-spm':"1000983"})[0].text.strip()

        try:
            # 页面上拉600，看到TabBarBox
            js = "window.scrollTo(0,600)"
            driver.execute_script(js)
        except WebDriverException:
            print('上拉寻找评论区时出现问题')

        time.sleep(random.uniform(1,2))
        # 点击累计评论TAB
        driver.find_element_by_xpath('//a[@href="#J_Reviews"]').click()
        time.sleep(random.uniform(1,2))
        print('成功点击累计评论TAB')

        while True:
            try:
                # rate-grid的正则表达式
                driver.find_element_by_xpath('//*[@id="J_Reviews"]/div/div[6]')
                print('已经成功加载到评论区信息')
                break
            except NoSuchElementException:
                print('评论信息中元素还未加载')
                continue


        allcmts = pd.DataFrame()
        
        count = 2
        while True:

            cmts = get_comments(driver.page_source)
            allcmts = pd.concat([allcmts,cmts],axis=0,ignore_index=True)

            try:
                js = "window.scrollTo(0,2500)"
                driver.execute_script(js)
                time.sleep(random.uniform(1,2))
                # 点击下一页
                nextpage = driver.find_element_by_css_selector('#J_Reviews > div > div.rate-page > div > a:last-child')
                driver.execute_script('arguments[0].click()',nextpage)
                print('\r已成功点击第' + str(count) +'页',end='')

            except NoSuchElementException:
                print('找不到翻页按钮,结束查询--(%s)' % title)
                break

            count += 1
            time.sleep(random.uniform(1,2))
        
        today = str(time.strftime('%Y-%m-%d',time.localtime(time.time())))
        allcmts.to_csv('%s-%s.csv'%(today,title),index=False,header=False,encoding='utf_8_sig')
        
    driver.close()

In [272]:
set = pd.read_csv('abc.csv',header=None)
urlist = [ i[0] for i in set.values.tolist()]

In [285]:
deal_comments_infos_fromurlist(urlist)

开始寻找评论from:https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.8.78fc62199gvL6p&id=587624116085&skuId=4015111008909&user_id=1664555011&cat_id=53636001&is_b=1&rn=5d0154e8231dec7275d44c19afbac8a7
成功点击累计评论TAB
已经成功加载到评论区信息
已成功点击第16页找不到翻页按钮,结束查询,(花花公子短袖T恤男士潮流体恤韩版半袖夏季打底衫男青年衣服上衣)
开始寻找评论from:https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.15.78fc62199gvL6p&id=587285450779&skuId=3991194400857&user_id=1679808547&cat_id=53636001&is_b=1&rn=5d0154e8231dec7275d44c19afbac8a7
成功点击累计评论TAB
已经成功加载到评论区信息
已成功点击第18页找不到翻页按钮,结束查询,(花花公子春夏季薄款休闲裤男士韩版潮流修身弹力束脚冰丝小脚裤子)
开始寻找评论from:https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.1.75ff74e7UdOIuF&id=585061276488&skuId=3987967185396&user_id=711882582&cat_id=53644001&is_b=1&rn=059a8dd60277e1886093b8a4ccb2e3fe
成功点击累计评论TAB
已经成功加载到评论区信息
已成功点击第3页找不到翻页按钮,结束查询,(Dickies袖部字母印花卫衣 男式胸前徽章上衣全棉圆领卫衣DK006465)
开始寻找评论from:https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.13.75ff74e7UdOIuF&id=587967088208&skuId=4008152908728&user_id=369351818&cat_

In [279]:
for url in urlist:
    cmtset = deal_comments_infos(url)
    today = str(time.strftime('%Y-%m-%d',time.localtime(time.time())))
    cmtset[1].to_csv('%s-%s.csv'%(today,cmtset[0]),index=False,header=False,encoding='utf_8_sig')

开始寻找评论from:https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.8.78fc62199gvL6p&id=587624116085&skuId=4015111008909&user_id=1664555011&cat_id=53636001&is_b=1&rn=5d0154e8231dec7275d44c19afbac8a7
成功点击累计评论TAB
已经成功加载到评论区信息
已成功点击第16页

找不到翻页按钮,结束查询,(花花公子短袖T恤男士潮流体恤韩版半袖夏季打底衫男青年衣服上衣)
开始寻找评论from:https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.15.78fc62199gvL6p&id=587285450779&skuId=3991194400857&user_id=1679808547&cat_id=53636001&is_b=1&rn=5d0154e8231dec7275d44c19afbac8a7
成功点击累计评论TAB
已经成功加载到评论区信息
已成功点击第18页

找不到翻页按钮,结束查询,(花花公子春夏季薄款休闲裤男士韩版潮流修身弹力束脚冰丝小脚裤子)
开始寻找评论from:https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.1.75ff74e7UdOIuF&id=585061276488&skuId=3987967185396&user_id=711882582&cat_id=53644001&is_b=1&rn=059a8dd60277e1886093b8a4ccb2e3fe
成功点击累计评论TAB
已经成功加载到评论区信息
已成功点击第3页

找不到翻页按钮,结束查询,(Dickies袖部字母印花卫衣 男式胸前徽章上衣全棉圆领卫衣DK006465)
开始寻找评论from:https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.13.75ff74e7UdOIuF&id=587967088208&skuId=4008152908728&user_id=36935181